In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from collections import Counter
import os

# Define transformations for the training and testing sets
transform = transforms.Compose([
    transforms.ToTensor()
])

# Load the training and testing datasets with the transformation applied
train_dataset = datasets.ImageFolder(root= "/kaggle/input/vibeid-a1/A1/train", transform=transform)
test_dataset = datasets.ImageFolder(root= "/kaggle/input/vibeid-a1/A1/test", transform=transform)

# Create dataloaders for the datasets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=2)

# # Verify the datasets and dataloaders
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

# Function to count the samples per class
def count_samples_per_class(dataset):
    class_counts = Counter()
    for _, label in dataset:
        class_counts[label] += 1
    return class_counts

# Count the number of samples per class in the training set
train_class_counts = count_samples_per_class(train_dataset)
print("\nTraining set class distribution:")
for class_label, count in train_class_counts.items():
    class_name = train_dataset.classes[class_label]
    print(f"Class '{class_name}' ({class_label}): {count} samples")

# Count the number of samples per class in the test set
test_class_counts = count_samples_per_class(test_dataset)
print("\nTest set class distribution:")
for class_label, count in test_class_counts.items():
    class_name = test_dataset.classes[class_label]
    print(f"Class '{class_name}' ({class_label}): {count} samples")


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms, datasets, models
import torchvision.transforms.functional as TF
import torch
import torch.nn as nn
import torchvision.models as models

model = models.resnet50(pretrained=False)
for param in model.parameters():
    param.requires_grad = True
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 100)  # Original fully connected layer
) # Assuming you have 2 classes
custom_resnet = model

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(custom_resnet.parameters())
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.01, patience=3, verbose=True)
# optimizer =  optim.Adam(custom_resnet.parameters())
# Train the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_resnet.to(device)
from torch.optim import lr_scheduler
# Define a learning rate scheduler
scheduler = lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)
num_epochs = 100
def train_and_test_model(model, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs):
    for epoch in range(num_epochs):
        # Training phase
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(train_loader)
        epoch_acc = correct / total

#         print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {epoch_loss:.4f}, Train Accuracy: {epoch_acc:.4f}")

        # Testing phase
        model.eval()  # Set the model to evaluation mode
        test_correct = 0
        test_total = 0
        test_loss = 0.0

        with torch.no_grad():  # Enable gradient computation during inference
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)

                outputs = model(images)
                loss = criterion(outputs, labels)
                test_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                test_total += labels.size(0)
                test_correct += (predicted == labels).sum().item()

        test_loss /= len(test_loader)
        test_accuracy = test_correct / test_total

        print(f"Epoch [{epoch+1}/{num_epochs}], Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}")

        # Update learning rate scheduler
        if scheduler is not None:
            scheduler.step(test_loss)

    print("Training completed!")


# Train and test the model
train_and_test_model(custom_resnet, train_loader, test_loader, criterion, optimizer, scheduler, device, num_epochs)

print("Training completed!")
